# **Section 1: `Load Modules`**

In [2]:
%matplotlib qt
from ipywidgets import widgets, Button, Layout
from ipyfilechooser import FileChooser
from IPython.display import display

from scipy.ndimage import gaussian_filter
import xanes_regtools as xr
import xanes_image_utils as xiu
import matplotlib.pyplot as plt
import h5py, tifffile, sys, os, shutil
import numpy as np
import skimage.morphology as skm
import scipy.ndimage as spn
from copy import deepcopy
from pathlib import Path
from importlib import reload
xr = reload(xr)
xiu = reload(xiu)
plt.rcParams['figure.figsize'] = [14, 8]

import scyjava_config
import imagej
try:
    scyjava_config.add_options('-Xmx20g')
    ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
    ijui = ij.ui()
    ijui.showUI()
except:
    print('ImageJ is already up running!')
    
from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')

def fiji_close_all(ips=None):
    if ips is None:
        imp = WindowManager.getCurrentImage()
        while imp is not None:
            imp.close()
            imp = WindowManager.getCurrentImage()
    else:
        for ip in ips:
            ip.close()
print('\033[04m\033[01m\033[34m     Section 1 finished     ') 

ImageJ is already up running!
     Section 1 finished     


# **Section 2: `User Inputs`**

In [70]:
# raw h5 file top directory
raw_h5_top_dir = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/' 
# recon_top_dir is where the recon folders are
recon_top_dir = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/' 
# trial registration save file name
trial_reg_save_file = 'xanes3D_trial_reg_results_29773_new.h5'
# these are [y_start, y_end, x_start, x_end]
roi = [370, 370+440, 660, 660+240]
# alignment section by section; sometime, you may see an abrupt misalignment between two chunks. You may try 
# different chunk_sz to correct such misalignment.
chunk_sz = 7
# scan id start
scan_id_s = 29773
# scan_id_end
scan_id_e = 29873
# reference scan
fixed_scan_id = 29823
# use_anchor: if fixed_scan will be used as an anchor for entire serires alignment
use_anchor = True

# if using a mask in alignment calculation
use_mask = True
# threshold in making a mask
mask_thres = 2e-3
# dilation width for enlarge the mask
mask_dilation = 10

# the index of the reconstructed slice in the reconstruction of the scan with fixed_scan_id
fixed_sli = 430
# a range of slice indices of the reconstructed slices in the reconstruction of 
# the scan with moving_scan_id
moving_sli_search_half_range = 30

# 'sr','pc', 'mpc'  
reg_method = 'mpc'
# ref_mode can be 'single', or 'neighbor' 
ref_mode = 'single'

trial_reg = None
print('\033[04m\033[01m\033[34m     Section 2 finished     ')   

Box(children=(Tab(children=(HBox(children=(VBox(children=(HTML(value='<span style="background-color:rgb(135,20…

     Section 2 finished     


In [72]:
help(widgets.Button.layout)

Help on InstanceDict in module ipywidgets.widgets.trait_types object:

class InstanceDict(traitlets.traitlets.Instance)
 |  An instance trait which coerces a dict to an instance.
 |  
 |  This lets the instance be specified as a dict, which is used
 |  to initialize the instance.
 |  
 |  Also, we default to a trivial instance, even if args and kwargs
 |  is not specified.
 |  
 |  Method resolution order:
 |      InstanceDict
 |      traitlets.traitlets.Instance
 |      traitlets.traitlets.ClassBasedTraitType
 |      traitlets.traitlets.TraitType
 |      traitlets.traitlets.BaseDescriptor
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  make_dynamic_default(self)
 |  
 |  validate(self, obj, value)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from traitlets.traitlets.Instance:
 |  
 |  __init__(self, klass=None, args=None, kw=None, **kwargs)
 |      Construct an Instance trait.
 |      
 |      This trait allows v

# **Section 3: `Calculate Displacement Amongst 3D Datasets`**

In [ ]:
moving_sli_search_s = fixed_sli - moving_sli_search_half_range
moving_sli_search_e = fixed_sli + moving_sli_search_half_range

recon_path_template = recon_top_dir + 'recon_fly_scan_id_{0}/recon_fly_scan_id_{0}_{1}.tiff'
ffn = recon_top_dir + 'recon_fly_scan_id_'+ str(fixed_scan_id) + \
      '/recon_fly_scan_id_' + str(fixed_scan_id) + '_' + str(fixed_sli).zfill(5) + '.tiff'
fixed  = tifffile.imread(ffn)[roi[0]:roi[1], roi[2]:roi[3]]
moving = np.ndarray([2*moving_sli_search_half_range, fixed.shape[0], fixed.shape[1]])

if use_mask:
    mask = skm.binary_dilation((fixed > mask_thres).astype(np.uint8),
                               np.ones([mask_dilation,mask_dilation])).astype(np.uint8)
    ijui.show(ij.py.to_java(mask))
    ijui.show(ij.py.to_java(fixed))
    ijui.show(ij.py.to_java(fixed*mask))

In [ ]:
fiji_close_all()

In [ ]:
reg = xr.regtools(dtype='3D_XANES', method=reg_method, mode='TRANSLATION')
reg.set_method(reg_method)
reg.set_ref_mode(ref_mode)
reg.cal_set_anchor(scan_id_s, scan_id_e, fixed_scan_id, raw_h5_top_dir=raw_h5_top_dir)
reg.set_chunk_sz(chunk_sz)
reg.set_roi(roi)
if use_mask:
    reg.use_mask = True
    reg.set_mask(mask)
reg.set_fixed_data(fixed)
reg.set_3D_recon_path_template(recon_path_template)
reg.set_saving(save_path=recon_top_dir, fn=trial_reg_save_file)
reg.xanes3D_sli_search_half_range = moving_sli_search_half_range
reg.xanes3D_recon_fixed_sli = fixed_sli
reg.reg_xanes3D_chunk()
print('\033[04m\033[01m\033[34m     Section 3 finished     ') 

In [ ]:
cnt = 0
for ii in reg.alignment_pair:
    print(cnt, '\t', np.array(ii), '\t', scan_id_s+np.array(ii))
    cnt += 1

In [ ]:
#trial_reg = None
try:
    for ip in ips:
        ip.close()
except:
    pass

if trial_reg is None:
    trial_reg = np.ndarray([2*moving_sli_search_half_range, roi[1]-roi[0], roi[3]-roi[2]])
    
alignment_pair_id = 3
fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
f = h5py.File(fn, 'r')
trial_reg[:] = f['/trial_registration/trial_reg_results/{0}/trial_reg_img{0}'.format(str(alignment_pair_id).zfill(3))][:]
f.close()

ips = []
ijui.show(ij.py.to_java(trial_reg))
imp1 = WindowManager.getCurrentImage()
imp1.setTitle(str(alignment_pair_id).zfill(3))
ips.append(imp1)

In [ ]:
fn = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773.txt'
with open(fn, 'r') as f:
    a = f.readlines()

b = []
for ii in a:
    if ii.split():
        #print(ii.split()[0])
        b.append([int(ii.split()[0]), int(ii.split()[1])+10])
        
fn = '/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773_new.txt'
np.savetxt(fn, b, fmt='%d')

In [ ]:
help(np.savetxt)

# **Section 4: `Align 3D Datasets`**

In [ ]:
from importlib import reload
import xanes_regtools as xr
xr = reload(xr)
import numpy as np
import h5py, os
shift = np.int32(np.genfromtxt('/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773_new.txt'))
fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
# this is the slice range for alignment after the shift values are determined
moving_sli_start = 480
moving_sli_end = 610

f = h5py.File(fn, 'r')
recon_top_dir = f['/trial_registration/data_directory_info/recon_top_dir'][()] 
recon_path_template = f['/trial_registration/data_directory_info/recon_path_template'][()] 
roi = f['/trial_registration/trial_reg_parameters/slice_roi'][:] 
reg_method = f['/trial_registration/trial_reg_parameters/reg_method'][()].lower()
ref_mode = f['/trial_registration/trial_reg_parameters/reg_ref_mode'][()].lower()
scan_id_s = f['/trial_registration/trial_reg_parameters/scan_ids'][0]
scan_id_e = f['/trial_registration/trial_reg_parameters/scan_ids'][-1]
fixed_scan_id = f['/trial_registration/trial_reg_parameters/fixed_scan_id'][()]
chunk_sz = f['/trial_registration/trial_reg_parameters/chunk_sz'][()]
eng_list = f['/trial_registration/trial_reg_parameters/eng_list'][:]
f.close()

reg = xr.regtools(dtype='3D_XANES', method=reg_method, mode='TRANSLATION')
reg.set_method(reg_method)
reg.set_ref_mode(ref_mode)
reg.cal_set_anchor(scan_id_s, scan_id_e, fixed_scan_id)
reg.eng_list = eng_list

reg.set_chunk_sz(chunk_sz)
reg.set_roi(roi)
reg.set_3D_recon_path_template(recon_path_template)
reg.set_saving(save_path=recon_top_dir, fn=os.path.basename(fn))
reg.apply_xanes3D_chunk_shift(shift, 410, 450, trialfn=fn, savefn=fn)
print('\033[04m\033[01m\033[34m     Section 4 finished     ') 

In [ ]:
try:
    for ip in ips:
        ip.close()
except:
    pass

fn = "/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/xanes3D_trial_reg_results_29773_new.h5"
sli_idx = 20
f = h5py.File(fn, 'r')
reg_img = f['/registration_results/reg_results/registered_xanes3D'][:, sli_idx, :, :]
f.close()

ips = []
ijui.show(ij.py.to_java(reg_img))
imp1 = WindowManager.getCurrentImage()
imp1.setTitle(str(sli_idx).zfill(3))
ips.append(imp1)

In [ ]:
shift = np.int32(np.genfromtxt('/media/XIAOUSB1/BNL_FXI_Sept2019_YujingBi_EnyuanHu/zshift_29773.txt'))
print(shift)

In [ ]:
print(shift)

In [ ]:
print(reg.eng_list)

In [ ]:
for key, item in reg.img_id_eng_dict.items():
    print(key, item)

In [ ]:
print(len(reg.abs_shift_dict))
print(len(reg.alignment_pair))
print(len(reg.shift_chain_dict))
cnt = 0
for key, item in reg.abs_shift_dict.items():
    print(cnt, ':\t', key, item)
    cnt += 1

In [ ]:
cnt = 0
for key, item in reg.shift_chain_dict.items():
    print(cnt, ':\t', key, item)
    cnt += 1

In [ ]:
print(len(reg.alignment_pair))
cnt = 0
for key, item in reg.alignment_pair:
    print(cnt, ':\t', key, item)
    cnt += 1

In [ ]:
print(reg.anchor)

In [ ]:
int(1.2)